In [1]:
%matplotlib inline


Doc2Vec Model
=============

Tuto : https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html




In [2]:
import os
import gensim

C:\Users\verny\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
from nltk.tokenize import word_tokenize

def read_corpus(directory, tokens_only=False):
    for filename in os.listdir(directory):
        with open(os.path.join(directory,filename), encoding="utf8") as f:
            for i, line in enumerate(f):
                tokens = word_tokenize(line, language="french")
                if tokens_only:
                    yield tokens
                else:
                    # For training data, add tags
                    yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [5]:
dir_path_train = os.path.join(os.path.dirname(os.path.realpath("../D2V_corpus/train")),"train")
dir_path_test = os.path.join(os.path.dirname(os.path.realpath("../D2V_corpus/test")),"test")

train_corpus = list(read_corpus(dir_path_train))
test_corpus = list(read_corpus(dir_path_test, tokens_only = True))

In [6]:
print(train_corpus[0])

TaggedDocument(['avant-propo', 'c', '’', 'être', 'à', 'vous', 'que', 'je', 'dédie', 'ce', 'livre', 'jeune', 'fille', 'qui', 'le', 'lire', 'ce', 'livre', 'fruit', 'de', 'mon', 'expérience', 'et', 'dans', 'lequel', 'je', 'me', 'suivre', 'efforcée', 'de', 'vous', 'peindre', 'le', 'vie', 'non', 'tel', 'que', 'vous', 'le', 'montre', 'votre', 'joyeux', 'imagination', 'qui', 'ne', 'le', 'sème', 'que', 'de', 'fleur', 'et', 'de', 'plaisir', 'mais', 'tel', 'qu', '’', 'lui', 'être', 'réellement', 'c', '’', 'être', '-', 'à-dir', 'remplie', 'de', 'devoir', 'sérieux', 'et', 'mêlée', 'de', 'pensée', 'grave', 'et', 'austère', 'avoir', 'vous', 'qui', 'êtes', 'assez', 'heureux', 'pour', 'vous', 'élever', 'sous', 'le', 'direction', 'sage', 'et', 'prudent', 'de', 'votre', 'mère', 'et', 'qui', 'de', 'suivre', 'son', 'conseil', 'à', 'vous', 'aussi', 'pauvre', 'enfant', 'que', 'le', 'ciel', 'avoir', 'privéer', 'de', 'ce', 'protection', 'saint', 'je', 'venir', 'vous', 'dire', 'déraciner', 'd

In [7]:
print(test_corpus[0])

['avant-propo', 'nous', 'connaître', 'un', 'homme', 'qui', 'au', 'milieu', 'de', 'le', 'fièvre', 'de', 'changement', 'et', 'd', '’', 'ambition', 'qui', 'travaille', 'notre', 'société', 'avoir', 'continué', 'd', '’', 'accepter', 'sans', 'révolte', 'son', 'humble', 'rôle', 'dans', 'le', 'monde', 'et', 'avoir', 'conservé', 'pour', 'ainsi', 'dire', 'le', 'goût', 'de', 'le', 'pauvreté', '.', 'sans', 'autre', 'fortune', 'qu', '’', 'un', 'petit', 'place', 'dont', 'il', 'voir', 'sur', 'ce', 'étroite', 'limite', 'qui', 'séparent', 'l', '’', 'aisance', 'de', 'le', 'misère', 'notre', 'philosophe', 'regarde', 'de', 'haut', 'de', 'son', 'mansarde', 'le', 'société', 'comme', 'un', 'mer', 'dont', 'il', 'ne', 'souhaite', 'point', 'le', 'richesse', 'et', 'dont', 'il', 'ne', 'craindre', 'pas', 'le', 'naufrage', 'tenir', 'trop', 'peu', 'de', 'place', 'pour', 'exciter', 'l', '’', 'envie', 'de', 'personne', 'il', 'dormir', 'tranquillement', 'enveloppé', 'dans', 'son', 'obscurité', '.', 'non

Training the Model
------------------

In [8]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [9]:
model.build_vocab(train_corpus)

In [10]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Testing the Model
-----------------

In [34]:
import random

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (480): «monsieur mm de staël avoir un grand et immense talent mais qui ne s ’ élève pas jusqu ’ au génie quelque chose de lourd mais de fort d ’ épai mais de puissant pèse à son pied quand lui vouloir quitter le terre il y avoir d ’ lui à jean-jacque son maître le différence qu ’ il y avoir de fer à l ’ acier – vous parler de son talent comme écrivain Monsieur parler moi un peu de le femme politique – sous ce rapport à mon avis monsieur répondit gilbert on donne à mme de staël beaucoup plus d ’ importance qu ’ lui n ’ en mérite depuis l ’ émigration de mounier et de lally son salon être le tribune de parti anglais semi-aristocratique avec le chambre comme lui être bourgeoise et très bourgeoise lui avoir le faiblesse d ’ adorer le grand seigneur lui admire le anglais parce qu ’ lui croire le peuple anglais un peuple éminemment aristocrate lui ne savoir pas l ’ histoire de l ’ angleterre lui ignore le mécanisme de son gouvernement de sorte qu ’ lui pren